## Introduction
This tutorial shows how to use tensorflow to train a neural network to mimic the $\neg (x_1 \oplus x_2)$ function. This function, abbreviated as XNOR, returns 1 only if $x_1$ is equal to $x_2$. The values are summarized in the table below:

$$
\begin{array}{c|c|c}
x_1 & x_2 & y \\ \hline
0 & 0 & 1 \\
0 & 1 & 0 \\
1 & 0 & 0 \\
1 & 1 & 1 \\
\end{array}
$$

In [16]:
import tensorflow as tf
import numpy as np

In [17]:
def CreateLayer(X, width):
    W = tf.get_variable("W", [X.get_shape()[1].value, width], tf.float32)
    b = tf.get_variable("b", [width], tf.float32)
    return tf.nn.sigmoid(tf.add(tf.matmul(X, W), b))

def CreateTrainigOp(model, learning_rate, labels):
    loss_op = tf.reduce_mean(tf.square(tf.subtract(model, labels)))
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss_op)
    return train_op, loss_op

In [21]:
g = tf.Graph()

with g.as_default():
  # Placeholders are fed data from the outside
  X = tf.placeholder(tf.float32, [None, 2], name="X")
  y = tf.placeholder(tf.float32, [None, 1], name="y")

  # A neural network consisting of two layers. The first layer
  # takex X, multiplies by weights W, adds biases b and applies
  # sigmoid function to produce z0. This is repeated for layer 2
  # except now z0 plays the role of the input. We use variable
  # scope to make weights and biases for both layers unique.
  with tf.variable_scope("layer1"):
      z0 = CreateLayer(X, 2)
  with tf.variable_scope("layer2"):
      z1 = CreateLayer(z0, 1)
  with tf.variable_scope("xnor"):
      training_op, loss_op = CreateTrainigOp(z1, 0.03, y) 
  init_op = tf.global_variables_initializer()
  saver = tf.train.Saver()

# The summary writer is going to be used to trace loss, output from the first
# layer (z0) and output from the second layer (z1)
writer = tf.summary.FileWriter("/tmp/xnor_log", g)
loss_summ = tf.summary.scalar("loss", loss_op)

# The input on which NN is trained. For, say [0, 0], we expect the NN to
# output something close to [1], etc.
X_train = np.array([[0, 0], [0, 1], [1, 0], [1, 1],])
y_train = np.array([[1], [0], [0], [1]])

sess = tf.Session(graph=g)
sess.run(init_op)
for step in xrange(5000):
    feed_dict = {X: X_train, y: y_train}
    sess.run(training_op, feed_dict=feed_dict)
    if step % 10 == 0:
        writer.add_summary(
            sess.run(loss_summ, feed_dict=feed_dict), step)
save_path = saver.save(sess, '/tmp/xnor.ckpt')
sess.close()
print "Model trained. Session saved in", save_path

Model trained. Session saved in /tmp/xnor.ckpt
